In [1]:
import numpy as np
from skimage.transform import resize
from experiments.cnn_sofm.load_cifar import load_data
import torchvision.models as models
import torch
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
cifar_10_dir = '../../data/cifar10'
(x_train, y_train), (x_test, y_test) = load_data(cifar_10_dir)

In [4]:
data_size = 200
x_train = x_train[:data_size]
y_train = y_train[:data_size]
del x_test
del y_test

In [5]:
resized_x = []
for i in range(data_size):
    resized_x.append(resize(x_train[i], (224, 224)))
x_train = torch.tensor(resized_x).reshape((-1, 3, 224, 224)).double().to(device)
x_train.shape
del resized_x

In [6]:
vgg = models.vgg16(pretrained=True).double().to(device)
vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
vgg.classifier[6] = torch.nn.Linear(4096, 10)

In [8]:
optimizer = torch.optim.Adam(vgg.classifier[6].parameters())

In [ ]:
epochs = 2
for epoch in range(epochs):
    optimizer.zero_grad()
    
    y_hat = vgg(x_train)
    loss = torch.nn.functional.binary_cross_entropy(y_hat, y_train)
    
    loss.backward()
    optimizer.step()
    
    print(loss)